In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from datetime import datetime

In [3]:
# Load the data
df = pd.read_csv('sales_data_trim.csv')

# Convert purchase_date to datetime
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='%d/%m/%Y %H:%M')

# Calculate total sales for each order
df['total_sales'] = df['purchase_qty'] * df['sales_price']

# Filter out records with material_code starting with '9'
df = df[~df['material_code'].astype(str).str.startswith('9')]

In [4]:
# Group by date and model, summing up total sales and quantity
daily_sales = df.groupby([df['purchase_date'].dt.date, 'model']).agg({
    'total_sales': 'sum',
    'purchase_qty': 'sum',
    'sales_price': 'mean'  # Using mean price for each product per day
}).reset_index()

In [ ]:
# Prepare features for regression
X = daily_sales[['model', 'sales_price']]
y = daily_sales['total_sales']

# One-hot encode the 'model' column
ct = ColumnTransformer([('encoder', OneHotEncoder(drop='first'), ['model'])], remainder='passthrough')
X_encoded = ct.fit_transform(X)

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_encoded, y)

In [3]:
# Function to predict total sales for a given combination
def predict_sales(models, prices):
    X_pred = pd.DataFrame({'model': models, 'sales_price': prices})
    X_pred_encoded = ct.transform(X_pred)
    return model.predict(X_pred_encoded)

Best combination of products and prices:
HD15#HD15: $10300.00
SV47#SV47: $3500.00
HP00#HP00: $1700.00
SV03#SV03: $4700.00
AM07#AM07: $8000.00
Predicted total sales: $132101.66


In [ ]:
# Get unique products and their max prices
unique_products = df.groupby('model')['sales_price'].max().reset_index()
unique_products = dict(zip(unique_products['model'], unique_products['sales_price']))

In [ ]:
# Calculate the daily average count of different products in 2024
df_2024 = df[df['purchase_date'].dt.year == 2024]
daily_product_count = df_2024.groupby(df_2024['purchase_date'].dt.date)['model'].nunique().mean()
daily_product_count = int(round(daily_product_count))

In [ ]:
best_combination = None
best_sales = 0

for _ in range(10000):  # Increased iterations for better results
    selected_models = np.random.choice(list(unique_products.keys()), size=daily_product_count, replace=False)
    selected_prices = [unique_products[model] for model in selected_models]
    
    predicted_sales = predict_sales(selected_models, selected_prices)
    
    if predicted_sales > best_sales:
        best_sales = predicted_sales
        best_combination = list(zip(selected_models, selected_prices))

print("Best combination of products and prices:")
for model, price in best_combination:
    print(f"{model}: ${price:.2f}")
print(f"Predicted total sales: ${best_sales[0]:.2f}")